In [2]:
import numpy as np
import pandas as pd
import nltk

In [9]:
#df=pd.read_csv('../train_task_b.csv')
df = pd.read_pickle('word_features_test.pickle')

In [7]:
df.head()

,paragraph_id,question_id,paragraph,question,answer
0,14754,60544,Первые упоминания о строении человеческого тел...,Где встречаются первые упоминания о строении ч...,в Древнем Египте
1,13859,1604,Телескоп имеет модульную структуру и содержит ...,Как называется корректирующая оптическая систе...,COSTAR
2,8310,30350,Критики теории Вегенера поставили во главу угл...,Какая теория была отвергнута после смерти Веге...,теория дрейфа материков
3,8766,53270,При нагревании кусочки янтаря становятся очень...,Чему не уступают по красоте изделия из прессов...,изделиям из монолитных камней
4,14719,75698,Нисходящие дифтонги со слабым гласным /i/ и /u...,На какие группы классифицируют дифтонги?,оральные и назальные


In [65]:
# Вот тут нужен на самом деле russian.pickle, но его придется делать самим
df['paragraph_sent']=df['paragraph'].map(nltk.sent_tokenize)

In [9]:
def answer_sent(row):
    sents = row['paragraph_sent']
    ans = row['answer'].lower().strip(' .')
    sent = next(filter(lambda s: ans in s.lower(), sents), None)
    return sents.index(sent) if sent else -1

In [66]:
# Где-то 1% не находит из-за неправильной разбивки предложений
df['answer_sent'] = df.apply(answer_sent, axis=1)

In [40]:
# Разбивка предложений на слова (наверно оптимальней было разбивать весь параграф?)
df['paragraph_sent_words'] = df['paragraph_sent'].map(lambda m: list(map(lambda s: set(nltk.word_tokenize(s)), m)))

In [51]:
# Разбивка вопросов на слова
df['question_words'] = df.question.map(lambda q: set(nltk.word_tokenize(q)))

In [12]:
def pred_sent(row):
    sent_words = row.paragraph_sent_words
    quest_words = row.question_words
    max = 0
    arg_max = -1
    for i,s in enumerate(sent_words):
        metric = len(s & quest_words)
        if metric>max:
            max=metric
            arg_max=i
    return arg_max

In [13]:
df['pred_sent'] = df.apply(pred_sent, axis=1)

In [85]:
correct=len(df[df.pred_sent==df.answer_sent])
total=len(df)
print("Result is {:.2f}%".format(correct/total*100))

Result is 80.27%


In [86]:
df.to_csv('word_features_test.csv')